In [35]:
 pip install bigtree

# $A^{3}_1$ Move Outcomes

In [36]:
from copy import deepcopy
from bigtree import Node, print_tree, tree_to_dot

# Player assignment
PLAYER_1 = 1
PLAYER_2 = 2

# House labels
house_labels = [
    "A^3_1", "A^3_2", "A^3_3",  # Player 1 houses
    "A^3_4", "A^3_5", "A^3_6"   # Player 2 houses
]

# Head indices
H_P1 = 6
H_P2 = 13

# Sowing orders for both players
sowing_order_p1 = [0, 1, 2, H_P1, 3, 4, 5]  # Player 1's sowing order
sowing_order_p2 = [3, 4, 5, H_P2, 0, 1, 2]  # Player 2's sowing order

# Initial state wherein there are 3 counters per house
initial_state = [3, 3, 3, 3, 3, 3, 0,  # Player 1 side + own head
                 3, 3, 3, 3, 3, 3, 0]  # Player 2 side + own head

def is_own_house(index, player):
    return (player == PLAYER_1 and 0 <= index <= 2) or (player == PLAYER_2 and 3 <= index <= 5)

# Reference for capture rule
def get_opposite_index(index):
    opposite_map = {0: 5, 1: 4, 2: 3, 3: 2, 4: 1, 5: 0,
                    7: 12, 8: 11, 9: 10, 10: 9, 11: 8, 12: 7}
    return opposite_map.get(index, None)

def get_sowing_order(player):
    return sowing_order_p1 if player == PLAYER_1 else sowing_order_p2

def get_head_index(player):
    return H_P1 if player == PLAYER_1 else H_P2

def get_opponent_head_index(player):
    return H_P2 if player == PLAYER_1 else H_P1

def valid_houses(player):
    return [0, 1, 2] if player == PLAYER_1 else [3, 4, 5]

def label(index):
    if 0 <= index <= 5:
        return house_labels[index]
    return ""

class TreeNode(Node):
    def __init__(self, state, player, move_sequence, score, last_player=None, parent=None):
        name = f"{'P1' if last_player == PLAYER_1 else 'P2'}: {move_sequence.strip()} (Score: {score})" if last_player else "Root"
        super().__init__(name=name, parent=parent)
        self.state = state
        self.player = player
        self.last_player = last_player
        self.move_sequence = move_sequence
        self.score = score

def sow_once(state, player, house_index):
    new_state = deepcopy(state)
    seeds = new_state[house_index]
    new_state[house_index] = 0
    order = get_sowing_order(player)
    opponent_head = get_opponent_head_index(player)

    index_in_order = order.index(house_index)
    while seeds > 0:
        index_in_order = (index_in_order + 1) % len(order)
        actual_index = order[index_in_order]
        if actual_index == opponent_head:
            continue
        new_state[actual_index] += 1
        seeds -= 1

    return new_state, actual_index

def perform_turn(state, player, initial_house_index):
    root = TreeNode(state, player, label(initial_house_index), (state[H_P1], state[H_P2]), last_player=player)

    def recurse(curr_state, curr_player, curr_sequence, node, house_index):
        if curr_state[house_index] == 0:
            return

        next_state, last_index = sow_once(curr_state, curr_player, house_index)
        sequence = curr_sequence + " " + label(last_index)

        # Handle capture
        if is_own_house(last_index, curr_player) and next_state[last_index] == 1:
            opp = get_opposite_index(last_index)
            if opp is not None and next_state[opp] > 0:
                captured = next_state[opp] + 1
                head = get_head_index(curr_player)
                next_state[head] += captured
                next_state[opp] = 0
                next_state[last_index] = 0

        # If last index is head, allow another choice that results to subturn
        if last_index == get_head_index(curr_player):
            sequence += " |"
            for h in valid_houses(curr_player):
                if next_state[h] > 0:
                    recurse(deepcopy(next_state), curr_player, sequence + " " + label(h), node, h)
            return

        # If landed on non-empty house, continue the turn
        if next_state[last_index] > 1:
            recurse(deepcopy(next_state), curr_player, sequence, node, last_index)
            return

        # Otherwise, switch to opponent
        next_player = PLAYER_2 if curr_player == PLAYER_1 else PLAYER_1
        child = TreeNode(next_state, next_player, sequence.strip(), (next_state[H_P1], next_state[H_P2]), last_player=curr_player, parent=node)

        # Only continue building the tree if not yet in terminal i.e. 18 counters in all heads.
        if next_state[H_P1] + next_state[H_P2] < 18:
            for h in valid_houses(next_player):
                if next_state[h] > 0:
                    recurse(deepcopy(next_state), next_player, label(h), child, h)

    recurse(state, player, label(initial_house_index), root, initial_house_index)
    return root

# Generate tree for Player 1 starting at A^3_1
A3_1 = 0
game_tree = perform_turn(initial_state, PLAYER_1, A3_1)

# Visualize the tree
print_tree(game_tree)


P1: A^3_1 (Score: (0, 0))
├── P1: A^3_1  | A^3_2 A^3_5 A^3_3 A^3_2  | A^3_1 A^3_3 A^3_4  | A^3_1 A^3_2 A^3_4 (Score: (7, 0))
│   ├── P2: A^3_4 A^3_5 A^3_1 (Score: (7, 1))
│   │   ├── P1: A^3_1 A^3_2 (Score: (7, 1))
│   │   │   └── P2: A^3_6 A^3_6 (Score: (7, 4))
│   │   │       ├── P1: A^3_2  | A^3_3 A^3_6 (Score: (9, 4))
│   │   │       │   ├── P2: A^3_4 A^3_6 A^3_1 (Score: (9, 5))
│   │   │       │   │   └── P1: A^3_1 A^3_2 (Score: (13, 5))
│   │   │       │   ├── P2: A^3_5  | A^3_4 A^3_6 A^3_2 (Score: (9, 6))
│   │   │       │   │   └── P1: A^3_2 A^3_3 (Score: (9, 6))
│   │   │       │   │       └── P2: A^3_5 A^3_6 (Score: (9, 8))
│   │   │       │   ├── P2: A^3_5  | A^3_6 A^3_1 (Score: (9, 6))
│   │   │       │   │   └── P1: A^3_1 A^3_2 (Score: (9, 6))
│   │   │       │   │       └── P2: A^3_4 A^3_6 (Score: (9, 6))
│   │   │       │   │           └── P1: A^3_2 A^3_3 (Score: (9, 6))
│   │   │       │   │               ├── P2: A^3_5 A^3_6 A^3_1 (Score: (9, 7))
│   │   │       │   │  

# $A^{3}_2$ Move Outcomes

In [37]:
from copy import deepcopy
from bigtree import Node, print_tree, tree_to_dot

# Player assignment
PLAYER_1 = 1
PLAYER_2 = 2

# House labels
house_labels = [
    "A^3_1", "A^3_2", "A^3_3",  # Player 1 houses
    "A^3_4", "A^3_5", "A^3_6"   # Player 2 houses
]

# Head indices
H_P1 = 6
H_P2 = 13

# Sowing orders for both players
sowing_order_p1 = [0, 1, 2, H_P1, 3, 4, 5]  # Player 1's sowing order
sowing_order_p2 = [3, 4, 5, H_P2, 0, 1, 2]  # Player 2's sowing order

# Initial state wherein there are 3 counters per house
initial_state = [3, 3, 3, 3, 3, 3, 0,  # Player 1 side + own head
                 3, 3, 3, 3, 3, 3, 0]  # Player 2 side + own head

def is_own_house(index, player):
    return (player == PLAYER_1 and 0 <= index <= 2) or (player == PLAYER_2 and 3 <= index <= 5)

# Reference for capture rule
def get_opposite_index(index):
    opposite_map = {0: 5, 1: 4, 2: 3, 3: 2, 4: 1, 5: 0,
                    7: 12, 8: 11, 9: 10, 10: 9, 11: 8, 12: 7}
    return opposite_map.get(index, None)

def get_sowing_order(player):
    return sowing_order_p1 if player == PLAYER_1 else sowing_order_p2

def get_head_index(player):
    return H_P1 if player == PLAYER_1 else H_P2

def get_opponent_head_index(player):
    return H_P2 if player == PLAYER_1 else H_P1

def valid_houses(player):
    return [0, 1, 2] if player == PLAYER_1 else [3, 4, 5]

def label(index):
    if 0 <= index <= 5:
        return house_labels[index]
    return ""

class TreeNode(Node):
    def __init__(self, state, player, move_sequence, score, last_player=None, parent=None):
        name = f"{'P1' if last_player == PLAYER_1 else 'P2'}: {move_sequence.strip()} (Score: {score})" if last_player else "Root"
        super().__init__(name=name, parent=parent)
        self.state = state
        self.player = player
        self.last_player = last_player
        self.move_sequence = move_sequence
        self.score = score

def sow_once(state, player, house_index):
    new_state = deepcopy(state)
    seeds = new_state[house_index]
    new_state[house_index] = 0
    order = get_sowing_order(player)
    opponent_head = get_opponent_head_index(player)

    index_in_order = order.index(house_index)
    while seeds > 0:
        index_in_order = (index_in_order + 1) % len(order)
        actual_index = order[index_in_order]
        if actual_index == opponent_head:
            continue
        new_state[actual_index] += 1
        seeds -= 1

    return new_state, actual_index

def perform_turn(state, player, initial_house_index):
    root = TreeNode(state, player, label(initial_house_index), (state[H_P1], state[H_P2]), last_player=player)

    def recurse(curr_state, curr_player, curr_sequence, node, house_index):
        if curr_state[house_index] == 0:
            return

        next_state, last_index = sow_once(curr_state, curr_player, house_index)
        sequence = curr_sequence + " " + label(last_index)

        # Handle capture
        if is_own_house(last_index, curr_player) and next_state[last_index] == 1:
            opp = get_opposite_index(last_index)
            if opp is not None and next_state[opp] > 0:
                captured = next_state[opp] + 1
                head = get_head_index(curr_player)
                next_state[head] += captured
                next_state[opp] = 0
                next_state[last_index] = 0

        # If last index is head, allow another choice that results to subturn
        if last_index == get_head_index(curr_player):
            sequence += " |"
            for h in valid_houses(curr_player):
                if next_state[h] > 0:
                    recurse(deepcopy(next_state), curr_player, sequence + " " + label(h), node, h)
            return

        # If landed on non-empty house, continue the turn
        if next_state[last_index] > 1:
            recurse(deepcopy(next_state), curr_player, sequence, node, last_index)
            return

        # Otherwise, switch to opponent
        next_player = PLAYER_2 if curr_player == PLAYER_1 else PLAYER_1
        child = TreeNode(next_state, next_player, sequence.strip(), (next_state[H_P1], next_state[H_P2]), last_player=curr_player, parent=node)

        # Only continue building the tree if not yet in terminal i.e. 18 counters in all heads.
        if next_state[H_P1] + next_state[H_P2] < 18:
            for h in valid_houses(next_player):
                if next_state[h] > 0:
                    recurse(deepcopy(next_state), next_player, label(h), child, h)

    recurse(state, player, label(initial_house_index), root, initial_house_index)
    return root

# Generate tree for Player 1 starting at A^3_1
A3_2 = 1
game_tree = perform_turn(initial_state, PLAYER_1, A3_2)

# Visualize the tree
print_tree(game_tree)


P1: A^3_2 (Score: (0, 0))
└── P1: A^3_2 A^3_4 A^3_2 (Score: (6, 0))
    ├── P2: A^3_6 A^3_3 A^3_1  | A^3_4 A^3_6 A^3_2 A^3_5 A^3_2 (Score: (6, 5))
    │   ├── P1: A^3_1 A^3_3 A^3_5 (Score: (7, 5))
    │   │   ├── P2: A^3_4 A^3_6 A^3_1 (Score: (7, 6))
    │   │   │   ├── P1: A^3_1 A^3_2 A^3_4 (Score: (8, 6))
    │   │   │   │   ├── P2: A^3_4 A^3_5 A^3_1 (Score: (8, 7))
    │   │   │   │   │   ├── P1: A^3_1 A^3_2 (Score: (8, 7))
    │   │   │   │   │   └── P1: A^3_3  | A^3_1 A^3_2 (Score: (9, 7))
    │   │   │   │   ├── P2: A^3_5  | A^3_4 A^3_5 (Score: (8, 7))
    │   │   │   │   └── P2: A^3_5  | A^3_6  | A^3_4 A^3_5 (Score: (8, 8))
    │   │   │   ├── P1: A^3_2  | A^3_1 A^3_2 (Score: (11, 6))
    │   │   │   └── P1: A^3_2  | A^3_3  | A^3_1 A^3_2 (Score: (12, 6))
    │   │   ├── P2: A^3_5 A^3_6 A^3_1 (Score: (7, 6))
    │   │   │   ├── P1: A^3_1 A^3_2 A^3_4 A^3_1 (Score: (10, 6))
    │   │   │   │   └── P2: A^3_5 A^3_6 (Score: (10, 6))
    │   │   │   ├── P1: A^3_2  | A^3_1 A^3_2 (Score:

# $A^{3}_3$ Move Outcomes

In [38]:
from copy import deepcopy
from bigtree import Node, print_tree, tree_to_dot

# Player assignment
PLAYER_1 = 1
PLAYER_2 = 2

# House labels
house_labels = [
    "A^3_1", "A^3_2", "A^3_3",  # Player 1 houses
    "A^3_4", "A^3_5", "A^3_6"   # Player 2 houses
]

# Head indices
H_P1 = 6
H_P2 = 13

# Sowing orders for both players
sowing_order_p1 = [0, 1, 2, H_P1, 3, 4, 5]  # Player 1's sowing order
sowing_order_p2 = [3, 4, 5, H_P2, 0, 1, 2]  # Player 2's sowing order

# Initial state wherein there are 3 counters per house
initial_state = [3, 3, 3, 3, 3, 3, 0,  # Player 1 side + own head
                 3, 3, 3, 3, 3, 3, 0]  # Player 2 side + own head

def is_own_house(index, player):
    return (player == PLAYER_1 and 0 <= index <= 2) or (player == PLAYER_2 and 3 <= index <= 5)

# Reference for capture rule
def get_opposite_index(index):
    opposite_map = {0: 5, 1: 4, 2: 3, 3: 2, 4: 1, 5: 0,
                    7: 12, 8: 11, 9: 10, 10: 9, 11: 8, 12: 7}
    return opposite_map.get(index, None)

def get_sowing_order(player):
    return sowing_order_p1 if player == PLAYER_1 else sowing_order_p2

def get_head_index(player):
    return H_P1 if player == PLAYER_1 else H_P2

def get_opponent_head_index(player):
    return H_P2 if player == PLAYER_1 else H_P1

def valid_houses(player):
    return [0, 1, 2] if player == PLAYER_1 else [3, 4, 5]

def label(index):
    if 0 <= index <= 5:
        return house_labels[index]
    return ""

class TreeNode(Node):
    def __init__(self, state, player, move_sequence, score, last_player=None, parent=None):
        name = f"{'P1' if last_player == PLAYER_1 else 'P2'}: {move_sequence.strip()} (Score: {score})" if last_player else "Root"
        super().__init__(name=name, parent=parent)
        self.state = state
        self.player = player
        self.last_player = last_player
        self.move_sequence = move_sequence
        self.score = score

def sow_once(state, player, house_index):
    new_state = deepcopy(state)
    seeds = new_state[house_index]
    new_state[house_index] = 0
    order = get_sowing_order(player)
    opponent_head = get_opponent_head_index(player)

    index_in_order = order.index(house_index)
    while seeds > 0:
        index_in_order = (index_in_order + 1) % len(order)
        actual_index = order[index_in_order]
        if actual_index == opponent_head:
            continue
        new_state[actual_index] += 1
        seeds -= 1

    return new_state, actual_index

def perform_turn(state, player, initial_house_index):
    root = TreeNode(state, player, label(initial_house_index), (state[H_P1], state[H_P2]), last_player=player)

    def recurse(curr_state, curr_player, curr_sequence, node, house_index):
        if curr_state[house_index] == 0:
            return

        next_state, last_index = sow_once(curr_state, curr_player, house_index)
        sequence = curr_sequence + " " + label(last_index)

        # Handle capture
        if is_own_house(last_index, curr_player) and next_state[last_index] == 1:
            opp = get_opposite_index(last_index)
            if opp is not None and next_state[opp] > 0:
                captured = next_state[opp] + 1
                head = get_head_index(curr_player)
                next_state[head] += captured
                next_state[opp] = 0
                next_state[last_index] = 0

        # If last index is head, allow another choice that results to subturn
        if last_index == get_head_index(curr_player):
            sequence += " |"
            for h in valid_houses(curr_player):
                if next_state[h] > 0:
                    recurse(deepcopy(next_state), curr_player, sequence + " " + label(h), node, h)
            return

        # If landed on non-empty house, continue the turn
        if next_state[last_index] > 1:
            recurse(deepcopy(next_state), curr_player, sequence, node, last_index)
            return

        # Otherwise, switch to opponent
        next_player = PLAYER_2 if curr_player == PLAYER_1 else PLAYER_1
        child = TreeNode(next_state, next_player, sequence.strip(), (next_state[H_P1], next_state[H_P2]), last_player=curr_player, parent=node)

        # Only continue building the tree if not yet in terminal i.e. 18 counters in all heads.
        if next_state[H_P1] + next_state[H_P2] < 18:
            for h in valid_houses(next_player):
                if next_state[h] > 0:
                    recurse(deepcopy(next_state), next_player, label(h), child, h)

    recurse(state, player, label(initial_house_index), root, initial_house_index)
    return root

# Generate tree for Player 1 starting at A^3_1
A3_3 = 2
game_tree = perform_turn(initial_state, PLAYER_1, A3_3)

# Visualize the tree
print_tree(game_tree)

P1: A^3_3 (Score: (0, 0))
└── P1: A^3_3 A^3_5 A^3_3 (Score: (6, 0))
    └── P2: A^3_6 A^3_3 (Score: (6, 1))
        ├── P1: A^3_1 A^3_5 (Score: (7, 1))
        │   └── P2: A^3_5 A^3_6 (Score: (7, 1))
        │       ├── P1: A^3_2 A^3_1 (Score: (11, 1))
        │       │   ├── P2: A^3_4 A^3_6 (Score: (11, 1))
        │       │   │   └── P1: A^3_3 A^3_5 A^3_2 (Score: (12, 1))
        │       │   │       ├── P2: A^3_4 A^3_5 (Score: (12, 3))
        │       │   │       │   └── P1: A^3_1 A^3_2 (Score: (12, 3))
        │       │   │       │       └── P2: A^3_6 A^3_1 (Score: (12, 4))
        │       │   │       │           └── P1: A^3_2 A^3_3 (Score: (12, 4))
        │       │   │       └── P2: A^3_6 A^3_1 A^3_3 (Score: (12, 2))
        │       │   │           └── P1: A^3_2  | A^3_3 A^3_4 A^3_6 (Score: (14, 2))
        │       │   │               ├── P2: A^3_5 A^3_6 A^3_1 (Score: (14, 3))
        │       │   │               │   └── P1: A^3_1 A^3_2 (Score: (14, 3))
        │       │   │       